In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2
# 多行输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [ ]:
from fastai import *
from fastai.text import *

In [ ]:
doc(Config)

- IMDB 精简数据

In [ ]:
path = untar_data(URLs.IMDB)
path
path.ls()

In [ ]:
BATCH = 32

In [ ]:
# data_lm = (TextList.from_folder(path)
#            #Inputs: all the text files in path
#             .filter_by_folder(include=['train', 'test']) 
#            #We may have other temp folders that contain text files so we only keep what's in train and test
#             .split_by_rand_pct(0.1)
#            #We randomly split and keep 10% (10,000 reviews) for validation
#             .label_for_lm()           
#            #We want to do a language model so we label accordingly, 自己就是自己的标签
#             .databunch(bs=BATCH))

In [ ]:
# data_lm.save('data_lm')

In [ ]:
data_lm = load_data(path, 'data_lm', bs=BATCH)

## 创建模型

In [ ]:
learn_lm = language_model_learner(data_lm, Transformer, drop_mult=0.3)

In [ ]:
learn_lm.loss_func
learn_lm.opt_func

In [ ]:
learn_lm.model

### 编码器

In [ ]:
encoder = get_model(learn_lm.model)[0]
encoder

- 生成假的评论，使用语言模型预测

In [ ]:
TEXT = "The color of the sky is"
N_WORDS = 40
N_SENTENCES = 2

In [ ]:
print("\n\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

- temperature 控制生成文本的随机性

In [ ]:
print("\n\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.1) for _ in range(N_SENTENCES)))

- 预训练数据集

In [ ]:
tr_itos = pickle.load(open(Config().model_path()/'transformer/itos_tfmer.pkl', 'rb'))

In [ ]:
tr_itos[:10]

In [ ]:
len(tr_itos), len(data_lm.vocab.itos) # wiki 词汇， IMDB 词汇

In [ ]:
unks[:16]

## 精调模型

In [ ]:
learn_lm.lr_find()

In [ ]:
learn_lm.recorder.plot(skip_end=10)

In [ ]:
learn_lm.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))

In [ ]:
learn_lm.save('transformer_fit_1')

In [ ]:
learn_lm=None
gc.collect()

- 保存

In [ ]:
data_lm = load_data(path, 'data_lm', bs=BATCH//2)

In [ ]:
learn_lm = language_model_learner(data_lm, Transformer, drop_mult=0.3)

In [ ]:
learn_lm.unfreeze()

In [ ]:
learn_lm.load('transformer_fit_1');

In [ ]:
learn_lm.fit_one_cycle(1, 1e-3, moms=(0.8,0.7))

In [ ]:
learn_lm.save('transformer_fit_2')

In [ ]:
learn_lm=None
gc.collect()

In [ ]:
learn_lm = language_model_learner(data_lm, Transformer, drop_mult=0.3)

In [ ]:
learn_lm.load('transformer_fit_2');

In [ ]:
TEXT = "i liked this movie because"
N_WORDS = 40
N_SENTENCES = 2

In [ ]:
print("\n\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
print("\n\n".join(learn_lm.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
learn_lm.save_encoder('fine_tuned_enc')

In [ ]:
learn_lm = None
gc.collect()

## 语义分类任务

In [ ]:
learn_c = None
gc.collect()

In [ ]:
path = untar_data(URLs.IMDB)
BATCH = 16

In [ ]:
# data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
#              #grab all the text files in path
#              .split_by_folder(valid='test')
#              #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
#              .label_from_folder(classes=['neg', 'pos'])
#              #label them all with their folders
#              .databunch(bs=BATCH))

In [ ]:
# data_clas.save('imdb_textlist_class')

In [ ]:
data_clas = load_data(path, 'imdb_textlist_class', bs=BATCH//4)

In [ ]:
data_clas.show_batch()

In [ ]:
learn_c = text_classifier_learner(data_clas, Transformer, drop_mult=0.5)

In [ ]:
learn_c.model

In [ ]:
learn_c.loss_func

In [ ]:
learn_c.load_encoder('fine_tuned_enc')

In [ ]:
learn_c.lr_find()
learn_c.recorder.plot()

In [ ]:
learn_c.fit_one_cycle(3, 5e-3, moms=(0.8, 0.7))

In [ ]:
learn_c.save('fine1')

In [ ]:
learn_c = None
gc.collect()